# PGE Creation
This notebook is the driver by which the *topsstack_hamsar_pge* notebook will be processed in order to be available as an SDS PGE. The notebook *submit_job* is then used to submit it to the SDS for execution.


### Do commit your source code changes first!

The build process retrieves source code for the PGE from the GitHub repository, so it is necessary to commit and push all changes before building/rebuilding the PGE.

### It also assumes you have a valid account on the cluster.

### 1. Initialize

When prompted for the HySDS host, include the protocol, e.g. https://soamc-mozart.jpl.nasa.gov

When prompted for "HySDS cluster authenticated", enter 'y' if the cluster requires a password to access.

In [ ]:
import os
from pathlib import Path
import otello
if not os.path.exists(f"{Path.home()}/.config/otello/config.yml"):
    otello.client.initialize()

### 2. Specify the repository and branch to use

This creates a handle to the Jenkins build process for your repository/PGE.

*Set "account" and "repository_name" to the correct value for the remote repository you created for this PGE.*

In [ ]:
account = "xing"
repository_name = "topsstack_hamsar"
repository_url = f"https://github.jpl.nasa.gov/{account}/{repository_name}.git"
branch = "devel"

print("Source code will be pulled from branch {1} of {0}".format(repository_url, branch))

ci = otello.CI(repo=repository_url, branch=branch)

### 3. Registering the repository/job-type with the build pipeline

The job-type/PGE must be registered with the build process before it can be built.

Existing registrations for the job-type/PGE are removed first, to ensure that the cached versions on active worker nodes are invalidated and replaced.

In [ ]:
try:
    if ci.check_job_exists():
        print("Job exists, unregistering")
        ci.unregister()
except Exception as e:
    print("Exception : {}, trying to register anyway".format(e))

ci.register()

### 4. Build the PGE

In [ ]:
ci.submit_build()

### 7. Wait for the build to resolve, then check whether the build was successful or not

In [ ]:
import time
move_on = False
while not move_on:
    try:
        ci.get_build_status()
        move_on = True
    except Exception as e:
        print("Waiting for build to start")
        time.sleep(10)
    
build_stat = ci.get_build_status()
print("Build in progress.", end="")

while (build_stat['building']):
    time.sleep(10)
    print(".", end="", flush=True)
    build_stat = ci.get_build_status()

if (build_stat['result'] == 'FAILURE'):
    print("\nBuild failed.")
else:
    print("\nBuild completed successfully")




Notes: (1) caputure meta info of docker image built?